# Quantum Tomography Overview

* **Author:** Gadi Aleksandrowicz (gadia@il.ibm.com)
* **Last Updated:** February 12, 2019



In [1]:
import numpy as np

## The General Theory

The goal of quantum tomography is obtain data on quantum systems by performing several different measurements. In Qiskit Ingis we are currently concerened with the following two tomography tasks:

1. **Quantum state tomograhpy**: Given a quantum state described by a density operator $\rho$, obtain a matrix representation for $\rho$.
2. **Quantum process tomograhpy**: Given a quantum channel $\mathcal{E}$, obtain a matrix representation (the **Choi matrix**) for $\mathcal{E}$.

In both cases we rely on the assumption that we have access to a large number of identical copies of the system and so can peform several different measures on it.

We can roughly split the tomography process to three stages:
1. Preperation: Add suitable initialization/measurement devices to the quantum system.
2. Experiment: Obtain measurement data from the quantum system.
3. Tomography: Use the obtained data to reconstruct the system's description.

Steps 1 and 2 are related to the quantum system being studied, whereas step 3 is a classical computation which can be carried out on standard computers.

## State Tomography Overview

Let $\rho$ be the density operator of a quantum state. If $\rho$ can be reliably reproduced, it can be subjected to several measurements with respect to different operators; this data can be used to reconstruct $\rho$ or a close approximation for it in several different methods.

### Example: 1-qubit reconstruction using the Pauli basis

Given the Pauli matrices $
I=\left(\begin{array}{cc}
1 & 0\\
0 & 1
\end{array}\right),
X=\left(\begin{array}{cc}
0 & 1\\
1 & 0
\end{array}\right),
Y=\left(\begin{array}{cc}
0 & -i\\
i & 0
\end{array}\right),
Z=\left(\begin{array}{cc}
1 & 0\\
0 & -1
\end{array}\right)$


In [2]:
I = np.array([[1,0],[0,1]])
X = np.array([[0,1],[1,0]])
Y = np.array([[0,-1j],[1j,0]])
Z = np.array([[1,0],[0,-1]])

It is easy to see they constitute an orthonormal basis for $M_2(\mathbb{C})$ with respect to the Hilbert-Schmidt inner product $\left\langle A,B\right\rangle =\frac{1}{2}\text{tr}\left(B^{\dagger}A\right)$

In [3]:
def HS_product(A,B):
    return np.trace(np.conj(B).T @ A)


And hence,

$$ \rho =\left\langle \rho,I\right\rangle I+\left\langle \rho,X\right\rangle X+\left\langle \rho,Y\right\rangle Y+\left\langle \rho,Z\right\rangle Z = $$

$$=\frac{\text{tr}\left(\rho\right)I+\text{tr}\left(X\rho\right)X+\text{tr}\left(Y\rho\right)Y+\text{tr}\left(Z\rho\right)Z}{2}$$

The values of $\text{tr}\left(X\rho\right), \text{tr}\left(Y\rho\right), \text{tr}\left(Z\rho\right)$ can be approximated by repeated measuring in the $X, Y$ and $Z$ bases. Since $\text{tr}\left(\rho\right)=1$ there is no need for additional measurements for the coefficient of $I$.

### Example: 1-qubit Linear inversion

The above method can be rephrased in more general form. First, any hermitian operator $H$ has a spectral decomposition of the form $H=\sum \lambda_i P_i$ where $\lambda_i$ is an eigenvalue of $H$ and $P_i$ is the projection operator to the corresponding eigenspace. For the hermitian operators $X,Y,Z$ whose eigenvalues are 1 and -1 we can therefore write

* $X = P^X_0-P^X_1$
* $Y = P^Y_0-P^Y_1$
* $Z = P^Z_0-P^Z_1$

Where



$$P^X_0=\frac{1}{2}\left(\begin{array}{cc}
1 & 1\\
1 & 1
\end{array}\right), P^X_1=\frac{1}{2}\left(\begin{array}{cc}
1 & -1\\
-1 & 1
\end{array}\right)$$

$$P^Y_0=\frac{1}{2}\left(\begin{array}{cc}
1 & -i\\
i & 1
\end{array}\right), P^Y_1=\frac{1}{2}\left(\begin{array}{cc}
1 & i\\
-i & 1
\end{array}\right)$$

$$P^Z_0=\left(\begin{array}{cc}
1 & 0\\
0 & 0
\end{array}\right), P^Z_1=\left(\begin{array}{cc}
0 & 0\\
0 & 1
\end{array}\right)$$

In the Ignis code, these matrices are defined in **tomography.fitters.utils.pauli_preparation_matrix**. We give an explicit definition here:

In [4]:
PX0 = 0.5*np.array([[1, 1], [1, 1]])
PX1 = 0.5*np.array([[1, -1], [-1, 1]])

PY0 = 0.5*np.array([[1, -1j], [1j, 1]])
PY1 = 0.5*np.array([[1, 1j], [-1j, 1]])

PZ0 = np.array([[1, 0], [0, 0]])
PZ1 = np.array([[0, 0], [0, 1]])

projectors = [PX0, PX1, PY0, PY1, PZ0, PZ1]

By Born's rule, $\text{tr}\left(P_{i}^{X}\rho\right)$ is the probability for the outcome $\left|i\right\rangle$ when measuring in the X-basis, and this probability can be estimated directly using repeated meausrements in the X-basis. The $Y$ and $Z$ bases are handled
similarily.

The computation $\text{tr}\left(P_{i}^{X}\rho\right)$ can be replaced by the scalar product $\vec{P}_i^X \cdot \vec{\rho}$ where $\vec{E}$ denotes the vector obtained from the operator $E$ by flattening its matrix (the result vector consists of the first row, then the second row etc.)

Now we can construct a matrix $$M=\left(\begin{array}{c}
\vec{P}_{0}^{X}\\
\vec{P}_{1}^{X}\\
\vec{P}_{0}^{Y}\\
\vec{P}_{1}^{Y}\\
\vec{P}_{0}^{Z}\\
\vec{P}_{1}^{Z}
\end{array}\right)$$

Such that $$M\vec{\rho}=\vec{p}=\left(\begin{array}{c}
p_{\left|0\right\rangle }^{X}\\
p_{\left|1\right\rangle }^{X}\\
p_{\left|0\right\rangle }^{Y}\\
p_{\left|1\right\rangle }^{Y}\\
p_{\left|0\right\rangle }^{Z}\\
p_{\left|1\right\rangle }^{Z}
\end{array}\right)$$

Is the equation relating the density operator to the observed probabilities.

In [5]:
M = np.array([p.flatten() for p in projectors])
M

array([[ 0.5+0.j ,  0.5+0.j ,  0.5+0.j ,  0.5+0.j ],
       [ 0.5+0.j , -0.5+0.j , -0.5+0.j ,  0.5+0.j ],
       [ 0.5+0.j ,  0. -0.5j,  0. +0.5j,  0.5+0.j ],
       [ 0.5+0.j ,  0. +0.5j,  0. -0.5j,  0.5+0.j ],
       [ 1. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  1. +0.j ]])

Since $M$ can be computed by knowing the operators used in the tomography, and the vector $\vec{p}$ of probabilities can be estimated using the tomography results, all that remains is to solve the equation $M\vec{\rho}=\vec{p}$ for $\vec{\rho}$. If the rank of $M$ is large enough this can be done by multiplying both sides by $M^\dagger$:

$M^\dagger M\vec{\rho} = M^\dagger \vec{p}$

$\vec{\rho} = (M^\dagger M)^{-1} M^\dagger \vec{p}$

In our example, we obtain the matrix 
$$(M^\dagger M)^{-1} M^\dagger = \left(\begin{array}{cccccc}
\frac{1}{6} & \frac{1}{6} & \frac{1}{6} & \frac{1}{6} & \frac{4}{6} & -\frac{2}{6}\\
\frac{1}{2} & -\frac{1}{2} & \frac{i}{2} & -\frac{i}{2} & 0 & 0\\
\frac{1}{2} & -\frac{1}{2} & -\frac{i}{2} & \frac{i}{2} & 0 & 0\\
\frac{1}{6} & \frac{1}{6} & \frac{1}{6} & \frac{1}{6} & -\frac{2}{6} & \frac{4}{6}
\end{array}\right)$$

In [6]:
M_dg = np.conj(M).T
linear_inversion_matrix = np.linalg.inv(M_dg @ M) @ M_dg

Multiplication by the linear inversion matrix performs the reconstruction stage described earlier to obtain the density operator.

### Example: 2-qubit Linear inversion

For multiple qubit systems the technique of linear inversion remains the same. The projector operators are tensor products of 1-qubit projectors: $6^n$ projectors in total, since we measure according to $3^n$ operators (tensor products of $X,Y,Z$) and each operator has two projectors.

In [7]:
import itertools
projectors_2 = [np.kron(p1, p2) for (p1, p2) in itertools.product(projectors, repeat = 2)]
M_2 = np.array([p.flatten() for p in projectors_2])
M_dg_2 = np.conj(M_2).T
linear_inversion_matrix_2 = np.linalg.inv(M_dg_2 @ M_2) @ M_dg_2

We will now attempt to reconstruct the Bell state $\frac{\left|00\right\rangle +\left|11\right\rangle }{\sqrt{2}}$ from simulated tomography results. First, we prepare a quantum circuit which generates this bell state from the input $\left|00\right\rangle$.

In [8]:
import qiskit 
from qiskit import QuantumRegister, QuantumCircuit

q2 = QuantumRegister(2)
bell = QuantumCircuit(q2)
bell.h(q2[0])
bell.cx(q2[0], q2[1])
bell.qasm()

'OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q0[2];\nh q0[0];\ncx q0[0],q0[1];\n'

We now use Ignis' **state_tomography_circuits** procedure which generates the $3^n$ circuits obtained by adding to the bell circuit a measurement according to each of our measurement operators (Pauli by default). Then we execute on a standard simulator.

In [9]:
from qiskit import Aer
import sys, os
sys.path.append(os.path.abspath(os.path.join('../qiskit_ignis')))
import tomography as tomo
qst_bell = tomo.state_tomography_circuits(bell, q2)
job = qiskit.execute(qst_bell, Aer.get_backend('qasm_simulator'), shots=5000)

Now we use the **tomography_data** procedure which converts the simulator output into the clear format used in tomography.

In [10]:
tomo_counts_bell = tomo.tomography_data(job.result(), qst_bell)
tomo_counts_bell

{('X', 'X'): {'11': 2494, '00': 2506},
 ('X', 'Y'): {'11': 1227, '01': 1269, '00': 1265, '10': 1239},
 ('X', 'Z'): {'11': 1248, '01': 1234, '00': 1250, '10': 1268},
 ('Y', 'X'): {'11': 1219, '01': 1232, '00': 1278, '10': 1271},
 ('Y', 'Y'): {'01': 2536, '10': 2464},
 ('Y', 'Z'): {'11': 1198, '01': 1228, '00': 1288, '10': 1286},
 ('Z', 'X'): {'11': 1308, '01': 1205, '00': 1260, '10': 1227},
 ('Z', 'Y'): {'11': 1263, '01': 1261, '00': 1227, '10': 1249},
 ('Z', 'Z'): {'11': 2513, '00': 2487}}

Now use **fitter_data** to explicitly extract the probability vector $\vec{p}$ and projector matrix $M$ that satisfy $M\vec{\rho} = \vec{p}$:

In [11]:
p, M, weights = tomo.fitter_data(tomo_counts_bell)

Now we use the linear inversion technique to reconstructo $\vec{\rho}$. Since we usually represent density matrices as matrices and not vectors, we use Numpy's **reshape** function to convert $\vec{\rho}$ into $\rho$.

In [12]:
M_dg = np.conj(M).T
linear_inversion_matrix = np.linalg.inv(M_dg @ M) @ M_dg
rho_bell = linear_inversion_matrix @ p
rho_bell = np.reshape(rho_bell, (4, 4))
print(rho_bell)

[[ 0.4979    +0.j          0.00073333+0.0015j      0.00616667+0.00093333j
   0.5       -0.0011j    ]
 [ 0.00073333-0.0015j      0.00123333+0.j          0.        -0.0005j
  -0.00743333+0.00293333j]
 [ 0.00616667-0.00093333j  0.        +0.0005j     -0.00123333+0.j
   0.00113333+0.0043j    ]
 [ 0.5       +0.0011j     -0.00743333-0.00293333j  0.00113333-0.0043j
   0.5021    +0.j        ]]


To check the quality of our solution, we compute the fidelity between the real quantum state (obtained via simulation by a simulator that can return state vectors) and our calculated $\rho$. The closer the fidelity to 1, the better.

In [13]:
from qiskit.quantum_info import state_fidelity
job = qiskit.execute(bell, Aer.get_backend('statevector_simulator'))
psi_bell = job.result().get_statevector(bell)
F_bell = state_fidelity(psi_bell, rho_bell)
print('Fit Fidelity linear inversion =', F_bell)

Fit Fidelity linear inversion = 1.0000000000000002


## Maximum Likelihood 

Linear inversion works perfectly on accurate data, but tomography data is never fully accurate. Two obvious obstacles are
1. Since the number of measurements is limited, we do not obtain the probability vector $\vec{p}$ but an approximation.
2. The measurement process might be noisy.

This may result in non-accurate or even self-contradicting $\vec{p}$, and the result of linear inversion might not be a density function at all (e.g. not nonnegative, or trace different than 1).

Since we want to solve the linear problem $A\vec{x}=\vec{p}$ for $x$, we can turn it into an optimization problem by attempting to minimize $\|A\vec{x}-\vec{p}\|_2$ while subjacting $x$ to additional constraints verifying this is indeed a density matrix. This is done by **state_cvx_fit**.

Another approach is to solve this optimization problem with no further constraints. The result might not be a density operator, i.e. positive semidefinite with trace 1; in this case the algorithm first rescales in order to obtain a density operator. This is done using **state_mle_fit**.

In [14]:
rho_cvx_bell = tomo.state_cvx_fit(p, M)
F_bell = state_fidelity(psi_bell, rho_cvx_bell)
print('Fit Fidelity CVX fit =', F_bell)

rho_mle_bell = tomo.state_cvx_fit(p, M)
F_bell = state_fidelity(psi_bell, rho_mle_bell)
print('Fit Fidelity MLE fit =', F_bell)

Fit Fidelity CVX fit = 0.9937579141994126
Fit Fidelity MLE fit = 0.9937579141994126


## Process Tomography Overview

A quantum process describes the evolution of a quantum system over time. While many processes can be described as function taking a state and returning another state (e.g. unitary gates), in general a process can induce uncertainty (e.g. because a measurement was performed on because of noise). Because of that, processes are modeled as a function $\mathcal{E}(\rho)=\rho^\prime$ on the space of density operators.

Process tomography aims to find a description of $\mathcal{E}$ from a set of measurements. Like state tomography, we measure with respect to several different bases. Process tomography also uses several different initial states, as opposed to state tomography where only $\left|0^n\right\rangle$ is used.

There are several different representations for a quantum process: To name a few there is the **Kraus operators** representation, the **Superoperator** representation, the **Choi matrix** representation and the **$\chi$ matrix** representation. In what follows we give explicit description for those representations. 

The Choi matrix representation has the advantage that its computation can be seen as a case of **state tomography** over a suitably chosen space. This enables us to use the same algorithms for both state and process tomography with only minor changes.

### Definitions

We denote by $X$ the state space. Usually in quantum computing it is a tensor product of 2-dimensional space spanned by $\left|0\right\rangle$ and $\left|1\right\rangle$; here we only assume it is a complex vector space of dimension $n$. Only elements of norm 1 in $X$ are considered to represent quantum states.

The space of all linear operators over $X$ is denoted by $L(X)$; every density operator is an element $\rho \in L(X)$ which is
1. Of trace 1, i.e. $\text{tr}(\rho) = 1$
2. Positive semidefinite, i.e. $\left\langle \psi \right|\rho \left|\psi\right\rangle \ge 0$ for all $\psi$. This is denoted by $\rho \ge 0$.

The space of all linear opeators over $L(X)$ is denoted by $T(X)$ (instead of using $L(L(X))$ which is more cubersome). A quantum process is an element of $T(X)$ mapping density operators to density operators. In order to do this, the process has to preserve the trace and positiveness of $\rho$; we require it to be **trace-preserving** and **completely positive**:

1. $\mathcal{E}\in T(X)$ is **trace-preserving** if $\text{tr}(\mathcal{E}(\rho))=\text{tr}(\rho)$ for every $\rho$.
2. $\mathcal{E}\in T(X)$ is **positive** if $\rho \ge 0$ implies $\mathcal{E}(\rho) \ge 0$. Moreover, $\mathcal{E}$ is **completely positive** if $\text{Id}\otimes \mathcal{E}$ is positive where $\text{Id}$ is the identity operator on any space of density operators.

#### Kraus operators

A common description of a quantum process $\mathcal{E}$ is via **Kraus operators**. A set $\left\{K_1, K_2, \dots, K_t\right\}$ such that $K_i \in L(X)$ with the additional constraint that $\sum_{i=1}^k K_i^\dagger K_i = I$
and the operation of $\mathcal{E}$ is given by a sum of conjugations: 
$$\mathcal{E}(\rho) = \sum_{i=1}^k K_i\rho K_i^\dagger$$

This is a common description of quantum processes. In case of a unitary operator $U$, the set $\left\{U\right\}$ is a Kraus operator representation of it. Measurements and noises are also commonly given via Kraus operators. Note however that the Kraus representation of a quantum process is not unique.

#### Superoperator

Since $\mathcal{E}$ is a linear operator over $L(X)$ then for any choice of basis for $L(X)$ it can be described as a matrix $\mathcal{S}$, such that $\mathcal{E}(\rho) = \mathcal{S}\cdot [\rho]$ where $[\rho]$ is the coordinate vector of $\rho$ over the same basis. $\mathcal{S}$ is called a **superoperator representation** of $\mathcal{E}$; it is dependent on the choice of basis for $L(X)$.

In practice $\rho$ is usually given by a matrix, and $[\rho]$ is obtained by **vectorization** of $\rho$; usually stacking the rows of $\rho$ one after another (different stacking methods result in different superoperator representations).

#### Choi matrix

Given a basis $\left\{\left|0\right\rangle, \left|1\right\rangle,\dots, \left|n-1\right\rangle\right\}$ of $X$, a basis for $L(X)$ consists of all the elements $\left|i\right\rangle\left\langle j\right|$ for $0\le i,j<n$. $\mathcal{E}$ is completely determined by its operation on these basis elements; the Choi matrix collects the results of those operations. Formally, the Choi matrix is defined by

$$\Lambda_\mathcal{E} = \sum_{i,j=0}^{n-1} \left|i\right\rangle\left\langle j\right| \otimes \mathcal{E}(\left|i\right\rangle\left\langle j\right|)$$

By representing $\left|i\right\rangle\left\langle j\right|$ as a matrix with 1 in the $ij$ entry and 0 everywhere else, we get the following explicit matrix representation of the Choi matrix:

$$\Lambda_\mathcal{E} = \left(\begin{array}{ccc}
\mathcal{E}\left(\left|0\right\rangle \left\langle 0\right|\right) & \cdots & \mathcal{E}\left(\left|0\right\rangle \left\langle n\right|\right)\\
\mathcal{E}\left(\left|1\right\rangle \left\langle 0\right|\right) & \cdots & \mathcal{E}\left(\left|1\right\rangle \left\langle n\right|\right)\\
\vdots & \ddots & \vdots\\
\mathcal{E}\left(\left|n\right\rangle \left\langle 0\right|\right) & \cdots & \mathcal{E}\left(\left|n\right\rangle \left\langle n\right|\right)
\end{array}\right)$$

#### $\chi$ matrix (process matrix)

The $\chi$ matrix representation is similar to the Kraus representation since it also gives $\mathcal{E}$ as a sum of operators acting on $\rho$ from the left and right; however, in this case the set of operators is not specific to $\mathcal{E}$ and can be any orthonormal basis for $L(X)$, and $\mathcal{E}$ is represented by the coefficients of the operators.

Formally, given a basis $\left\{\sigma_0, \dots \sigma_{n^2-1}\right\}$ for $L(X)$, $\mathcal{E}$ can be represented by

$$\mathcal{E}(\rho) = \sum_{i,j=0}^{n^2-1}\chi_{ij}\sigma_i\rho\sigma_j^\dagger$$

The values $\chi_{ij}$ are scalars uniquely determined by $\mathcal{E}$ and the choice of basis. The $n^2 \times n^2$ matrix $\chi$ is called the $\chi$ matrix of the process.

### Process tomography with the Choi matrix

Given a quantum process $\mathcal{E}$ with a Choi matrix $\Lambda_\mathcal{E}$, the evolution of a state $\rho$ is 
$$\mathcal{E}(\rho) = \text{Tr}_1\left[(\rho^t\otimes\mathbb{1})\Lambda_\mathcal{E}\right]$$

Where $\text{Tr}_1$ denotes **partial trace**: $\text{Tr}_1[A\otimes B] = \text{Tr}_1[A]B$ and $\mathbb{1}$ is the identity operator on $X$. The above equation is obtained in a straightforward manners from the definitions and the fact that $(A\otimes B)(C\otimes D)=(AC)\otimes(BD)$.

Using the above equation it is possible to show that the act of measuring the outcome of $\mathcal{E}$ on some initial $\rho$ with some projector $P$ can be seen as the act of measuring $\Lambda_\mathcal{E}$ (when it is considered as a **state** in the space $L(X)$) with a measurement operator $\overline{\rho}\otimes P$.

This gives rise to the following algorithm for process tomography:

1. Obtain a set of initial states $\left\{\rho_1, \dots \rho_{k}\right\}$ and a set of projectors $\left\{P_1, \dots P_{t}\right\}$ such that the set of all projectos $\Pi_{i,j} = \overline{\rho_i}\otimes P_j$ is tomographically complete for $L(X)$.
2. Obtain measurements of $\Lambda_\mathcal{E}$ with $\Pi_{i,j}$ by initializing a system to $\rho_{i}$, applying $\mathcal{E}$ (e.g. via a simulator or the quantum computer to check) and measuring via $P_j$.
3. Pass the results and the description of $\Pi_{i,j}$ to an algorithm for state tomography and obtain $\Lambda_\mathcal{E}$. 